# Data transformation, cleaning and loading with Python

## Astronomy Data Set
We are considering an astronomy data set this week from the Australia Telescope 20GHz Survey  (AT20G) from 2008. In Piazza (Resources section), there is an Excel sheet available with an excerpt of the original data set, as well as an readme document which explains the scenario and all the different attributes in the Excel workbook. We also have made available the content of the Excel workbook as a set of five different CSV files. Please upload those CSV files to Jupyter first. 

**Important:** Make sure that the naming of all the files is as follows:
 1. 04-at20g-short-epochs.csv
 2. 04-at20g-short-frequencies.csv
 3. 04-at20g-short-main-catalogue.csv
 4. 04-at20g-short-observations.csv
 5. 04-at20g-short-variabilities.csv


## EXERCISE 1: DB Creation and Loading Data

The following assumes a bit of background on SQL, in particular on its core commands to create new tables and to retrieve data:

 SQL Command   |  Meaning
 --------------|------------
 CREATE TABLE *T* (...)      | creates a new table *T*; list the attributes in brackets in the form  <tt>attribute type</tt>
 DROP TABLE *T*              | if needed - removes an existing table *T*
 INSERT INTO *T* VALUES (..) | inserts a new row into table T
 DELETE FROM *T*             | deletes *all* rows from table *T*
 SELECT COUNT(\*) FROM *T*   | count how many tuples are stored in table *T*
 SELECT \* FROM *T*          | list the content of table *T*

You can learn more background on these SQl commands in the SQL tutorial part in Grok.

### Database Creation, Part 1: PostgreSQL
First, we  need to create a target table in our PostgreSQL database. Relational databases work 'schema first': We first have to create a schema which defines the layout and typing of the database tables before we can load and query any data in a relational system. 

Looking at the source data, we assume two integer columns, the first one being unique.

The next step we try to **do outside Python in a Jupyter shell** (we will later show how to do it inside Python, but sometimes shell work is faster):

Go to the Jupyter start page and open a Terminal in Jupyter using the 'New' menu:

![New Terminal](http://www.it.usyd.edu.au/~roehm/teaching/comp5310/screenshot_postgres-terminal-new.png "New Terminal")

A new Terminal window now open.

Here you can directly connect to the postgresql database using the 'psql' command.

Enter 'psql':
![PSQL prompt](http://www.it.usyd.edu.au/~roehm/teaching/comp5310/screenshot_postgresql_terminal-prompt.png "PSQL prompt")


Then on the psql prompt, give the following SQL create table statement:

    CREATE TABLE IF NOT EXISTS Frequency (
       frequency INTEGER PRIMARY KEY,
        abbrev    SMALLINT
    );
 
You can verify whether you created the table correctly with the \d command:

     \d
     \d Frequency

You should see the following:
![Frequency Table Schema](http://www.it.usyd.edu.au/~roehm/teaching/comp5310/screenshot_postgresql_table-frequency "Frequency Table Schema")

### CSV File Loading, Part 1: Frequency Data 
Next we want to load data from an external CSV file.
We will use psql's **\copy** command for this.

**Prerequisites:** Make sure, that you have uploaded the CSV files and that the filenames are as specified at the top of this notebook.

To load data from a CSV file into a relational database, we have to tell the system
 - into which table to load the data ('Frequency')
 - which attributes to expect; this is optional, but if you are unsure whether the order in the CSV columns matches the order of attributes in a table, it is best to specify it here. Basically in our example, we specify that we will read 'frequency' and 'abbrev' values from the CSV file in this order.
 - from which file to load the data; be sure to use **\copy** rather than just COPY so that you can use a relatibe filename relative to the current directory
 - which format to expect (CSV) and whether there is a header row that should be ignored (yes, it is - HEADER)

So with all this, the final command to load the frequency table is as follows.
Please type into the psql prompt at the terminal:
 
    \copy Frequency (frequency,abbrev) FROM '04-at20g-short-frequencies.csv' WITH CSV HEADER

### Database Creation, Part 2: Variabilities Table

Psql's <tt>\copy</tt> command is quite useful - as long as table and CSV files directly match, and as long as the CSV file's content is in good shape. Otherwise it soon reaches its limits.

For example lets try using <tt>\copy</tt> for loading the next file with variability data.
We first have to create a new table again. Enter the following SQL command at the psql prompt to create a new table:

    CREATE TABLE IF NOT EXISTS Variabilities (
              source   VARCHAR(20) PRIMARY KEY,
              oct04_flux FLOAT,
              oct04_err  FLOAT,
              oct05_flux FLOAT,
              oct05_err  FLOAT,
              apr06_flux FLOAT,
              apr06_err  FLOAT
    );
    
Check whether the table has been created correctly:

        \d Variabilities
        
You should see the following:
![Variability Table Schema](http://www.it.usyd.edu.au/~roehm/teaching/comp5310/screenshot_postgresql_table-variability "Variability Table Schema")

### CSV File Loading, Part 2: Variability Data 

Next try to load the corresponding CSV data:

    \copy Variabilities FROM '04-at20g-short-variabilities.csv' WITH CSV HEADER

Unfortunately, this will result in an error:

![Variability Loading Error](http://www.it.usyd.edu.au/~roehm/teaching/comp5310/screenshot_postgresql_load-variabilities-error.jpg "Variability Loading Error")

What has happened?

If you look at the raw '04-at20g-short-variabilities.csv' CSV file, you will find quite a few 'x' entries for some of the variability measurements. These 'x' entries are a problem for <tt>\copy</tt> because it does not know how to convert those to valid FLOAT numbers, or whether it should replace them with NULL. 

The only option we have with <tt>\copy</tt> is at this moment to define that those 'x' entries should be replace with the special NULL value of SQL. You can do so with the **NULL** option:

    \copy Variabilities FROM '04-at20g-short-variabilities.csv' WITH CSV HEADER NULL 'x'
    
If you run this command, you tell <tt>\copy</tt> to ignore all 'x' in the CSV file and replace them with a NULL entry instead. This works fine now. You can check you success with the following SQL query after the correct <tt>\copy</tt> command:

    SELECT * FROM Variabilities;
    
![Variability Loading Ok](http://www.it.usyd.edu.au/~roehm/teaching/comp5310/screenshot_postgresql_load-variabilities-OK.jpg "Variability Loading Ok")

This looks already quite promising, but note a few shortcomings of this approach with <tt>\copy</tt>:
- The CSV columns have to match 1:1 the table schema in the database
- We can replace mismatching entries with NULL, but nothing else (eg. no NaN for not-a-number)
- We can only replace one well-defined data mismatch, not multiple
- There is no mechanism to call a user-defined conversion function for such data where we need to convert it first

Basically <tt>\copy</tt> is a very good and fast approach to load well-formed data, such as a previous database export, into a PostgreSQL database. It does not help us if the data is not so well behaved, or if we have to split and load data into separate tables.

# STOP PLEASE. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.

## EXERCISE 2: Loading Data into a Database via Python

**Next we are back to Python.**

We continue with the same Python environment than last week: the `DictReader` from the `csv` module which support reading and writing of files in comma-separated values (CSV).

Make sure that you have uploaded the 'at20g-short-frequencies.csv' CSV file into Jupyter.
We will first load the content of this file into Python with the same  csv.DictReader()  mechanism than last week:

In [ ]:
import csv
import pprint
data_frequencies = list(csv.DictReader(open('04-at20g-short-frequencies.csv')))
pprint.pprint(data_frequencies[0])

We will need to execute some SQL statements against the database. As we will have to do so multiple times, we write a dedicated function for executing an arbitrary SQL statement, where we do not expect any result. This handles then also all failures and using psycopg2's 'with' statements also the transaction processing of the database. Below's code will for example automatically commit our SQL statements, as well as rollback if there was any error.

In [ ]:
def pgexec( conn, sqlcmd, args, msg, silent=False ):
   """ utility function to execute some SQL statement
       can take optional arguments to fill in (dictionary)
       error and transaction handling built-in """
   retval = False
   with conn:
      with conn.cursor() as cur:
         try:
            if args is None:
               cur.execute(sqlcmd)
            else:
               cur.execute(sqlcmd, args)
            if silent == False: 
                print("success: " + msg)
            retval = True
         except Exception as e:
            if silent == False: 
                print("db error: ")
                print(e)
   return retval

Above's function ignores any query result from the SQL statement executed. To be able to also query the content of the database, we introduce another utility function which again encapsulates all error and transaction handling.

In [ ]:
def pgquery( conn, sqlcmd, args, silent=False ):
   """ utility function to execute some SQL query statement
       can take optional arguments to fill in (dictionary)
       will print out on screen the result set of the query
       error and transaction handling built-in """
   retval = False
   with conn:
      with conn.cursor() as cur:
         try:
            if args is None:
                cur.execute(sqlcmd)
            else:
                cur.execute(sqlcmd, args)
            if silent == False:
                for record in cur:
                    print(record)
            retval = True
         except Exception as e:
            if silent == False:
                print("db read error: ")
                print(e)
   return retval

For larger data sets, the following would normally be executed as a stand alone Python program on a shell.
First, you need to establish a connection to the postgresql database. 
Please edit the database name in below's code to match your Jupyter login.

In [ ]:
import psycopg2

DATABASENAME = '<your login name>'  # please replace with your own Jupyter login

In [ ]:
try: 
    conn = psycopg2.connect(database=DATABASENAME)
    print('connected')
except Exception as e:
    print("unable to connect to the database")
    print(e)

Next let's check whether this has all worked fine by querying our PostgreSQL database again.
You of course can go back to the Terminal page and in pgsql simply type   SELECT * FROM Frequency

Or we do it here in Python again.
Using our previous pgquery() function, we query the new Frequency table and simply print out all tuples found.

In [ ]:

# check content of Frequency table
query_stmt = "SELECT * FROM Frequency"
print(query_stmt)
pgquery (conn, query_stmt, None)

# cleanup... so that we are not running out of connections on our PostgreSQL server
conn.close()

## Your Task: Data Loading

Try to create and load the Variabilities table.
We have provided a CSV file with the corresponding data of measurement fluctuations for a small subset of the astronomical sources. Your task is three-fold:
 1. Create a matching 'Variability' table to hold the CSV data
 2. Load the content of the csv file into a local 'data_variabilities' dictionary in Python
 3. Load the data from the 'data_variabilities' dictionary into your PostgreSQL table
 4. Query and print its content

In [ ]:
import csv
import pprint
data_variabilities = list(csv.DictReader(open('04-at20g-short-variabilities.csv')))
pprint.pprint(data_variabilities[0])

Now let's load our previous data.
Important: whenever you use this approach, make sure that the header line of your CSV file has no spaces in its column titles and also no quotes. Otherwise, the csv.DictReader might be fine to read it, but not the psycopg2's cursor.execute() function. We are using named placeholders in out INSERT statement below (eg. '%(frequency_mhz)s' ) which expects to put a string (%s) into that place of the INSERT statement as been found in the given dictionary for the execute() call with the key 'frequency_mhz'.

In [ ]:
try: 
    conn = psycopg2.connect(database=DATABASENAME)
    print('connected')
except Exception as e:
    print("unable to connect to the database")
    print(e)
    
# 1st ensure that the schema is in place
#
table_schema = """CREATE TABLE IF NOT EXISTS Variabilities (
              source   VARCHAR(20) PRIMARY KEY,
              oct04_flux FLOAT,
              oct04_err  FLOAT,
              oct05_flux FLOAT,
              oct05_err  FLOAT,
              apr06_flux FLOAT,
              apr06_err  FLOAT
            )"""
pgexec (conn, table_schema, None, "Create Table Variabilities")

# 2nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO Variabilities(source,oct04_flux,oct04_err,oct05_flux,oct05_err,apr06_flux,apr06_err)
                     VALUES (%(source)s, %(oct04_flux)s, %(oct04_err)s, %(oct05_flux)s, %(oct05_err)s, %(apr06_flux)s, %(apr06_err)s)"""
for row in data_variabilities:
    pgexec (conn, insert_stmt, row, "row inserted")


In [ ]:
# check content of Frequency table
query_stmt = "SELECT * FROM Variabilities"
print(query_stmt)
pgquery (conn, query_stmt, None)

conn.close()

# STOP PLEASE. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.

## EXERCISE 3: Data Cleaning

### Data Cleaning
We re-use the iter_clean() function from last week.

In [ ]:
import warnings
import numpy as np
DEFAULT_VALUE = np.nan
def iter_clean(data, column_key, convert_function, default_value):
    for row in data:
        old_value = row[column_key]
        new_value = default_value
        try:
            new_value = convert_function(old_value)
        except (ValueError, TypeError):
            warnings.warn('Replacing {} with {} in column {}'.format(
                row[column_key], new_value, column_key))
        row[column_key] = new_value
        yield row

# this conversion strips any leading or trailing spaces from the 'frequency_mhz' values
data_frequencies = list(iter_clean(data_frequencies, 'frequency_mhz', str.strip, DEFAULT_VALUE))

# the following converts the two frequency columns to int values - or NaN
data_frequencies = list(iter_clean(data_frequencies, 'frequency_mhz', int, DEFAULT_VALUE))
data_frequencies = list(iter_clean(data_frequencies, 'abbreviated_frequency_ghz', int, DEFAULT_VALUE))

## Your Task: Data Cleaning

Use above's  iter_clean()  iterator function to clean the other give data set too.
 1. Clean the  'data_variabilities'  data set
 2. Reload the 'data_variabilities'  dictionary into your database
 3. Query the 'Variabilities' table - which difference do you see?
 
 4. If you have time: Do all of the above (reading - cleaning - loading) also for the 'Measurements.csv' data set

Note: You might encounter a few warning and error messages.
   - If a connection is closed, you have to open the databse connection again first
   - If the iter_clean() function returns a warning that some string was replaced with NaN, as long as this is indeed a number attribute, you are Ok to ignore this message. It just tells you that it is doing what it is supposed to do.
   - If you try to insert data into an already existing table with data inside, you might get 'duplicate primary key' error messages. Again, you can ignore those for the moment.
   - If you want to see who much data is already in your table, use the following SQL query:
     -  SELECT COUNT(*) FROM Variability;

In [ ]:
data_variabilities = list(iter_clean(data_variabilities, 'oct04_flux', float, DEFAULT_VALUE))
data_variabilities = list(iter_clean(data_variabilities, 'oct04_err',  float, DEFAULT_VALUE))
data_variabilities = list(iter_clean(data_variabilities, 'oct05_flux', float, DEFAULT_VALUE))
data_variabilities = list(iter_clean(data_variabilities, 'oct05_err',  float, DEFAULT_VALUE))
data_variabilities = list(iter_clean(data_variabilities, 'apr06_flux', float, DEFAULT_VALUE))
data_variabilities = list(iter_clean(data_variabilities, 'apr06_err',  float, DEFAULT_VALUE))

Next lets insert the cleaned data into the existing table.
Note that we had previously already 22 rows loaded into the 'Variability' table. Hence we will get 22 'primary key violation' errors. You can safely ignore those errors for the moment. The important point is that the three additional entries with missing data are now inserted too, so that at the end the Variability table contains 25 rows.

In [ ]:
try: 
    conn = psycopg2.connect(database=DATABASENAME)
    print('connected')
except Exception as e:
    print("unable to connect to the database")
    print(e)

# check content of Frequency table
query_stmt = "SELECT COUNT(*) FROM Variabilities"
print(query_stmt)
pgquery (conn, query_stmt, None)

# Try again to load data - 3 more tupels hould now go in
insert_stmt = """INSERT INTO Variabilities(source,oct04_flux,oct04_err,oct05_flux,oct05_err,apr06_flux,apr06_err)
                     VALUES (%(source)s, %(oct04_flux)s, %(oct04_err)s, %(oct05_flux)s, %(oct05_err)s, %(apr06_flux)s, %(apr06_err)s)"""
for row in data_variabilities:
    pgexec (conn, insert_stmt, row, "row inserted")

# check content of Frequency table
query_stmt = "SELECT COUNT(*) FROM Variabilities"
print(query_stmt)
pgquery (conn, query_stmt, None)

conn.close();


In [ ]:
# Finally, also loading the other CSV files
data_maincatalogue = list(csv.DictReader(open('04-at20g-short-main-catalogue.csv')))
#pprint.pprint(data_maincatalogue[0])

data_epochs = list(csv.DictReader(open('04-at20g-short-epochs.csv')))
#pprint.pprint(data_epochs[0])

data_observations = list(csv.DictReader(open('04-at20g-short-observations.csv')))
#pprint.pprint(data_observations[0])

# STOP PLEASE. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.

## EXERCISE 3: Data Modelling & DB Creation
In the next exercise we look at modelling the data correctly for a normalised relational star schema. We will first do this as a break-out exercise, then after about 10 minutes collect our findings and discuss an example solution for the given astronomy data model.

## Your Task: Data Modelling
With your colleagues at the same table, have a look at the complete Excel workbook and discuss how you would model its content using either UML or the Entity Relationship Model. 

## EXERCISE 4: Database Creation
After we discussed the model, we will give put an example solution.
The next step is to create the corresponding SQL schema in your PostgreSQL database.

### Your Task: DB Creation in PostgreSQL
Create the corresponding tables in PostgreSQL which follow from the data model.

<pre>
CREATE TABLE IF NOT EXISTS FrequencyBand (
   band       INTEGER,
   frequency1 FLOAT,
   frequency2 FLOAT,
   CONSTRAINT FrequencyBandPK PRIMARY KEY(band),
   CONSTRAINT FrequencyBandU1 UNIQUE (frequency1),
   CONSTRAINT FrequencyBandU2 UNIQUE (frequency2)
   /* further potential integrity constraints: frequencyX >0 and < ... */
   /* alternative: FrequencyBand(band PK, frequency PK) */
);

CREATE TABLE IF NOT EXISTS  Galaxy (
   gid  VARCHAR(20),
   ra   VARCHAR(11),
   dec  VARCHAR(11),
   CONSTRAINT GalaxyPK PRIMARY KEY(gid)
);

CREATE TABLE IF NOT EXISTS  TelescopeConfig (
   configID   INTEGER,
   minDec     INTEGER,
   maxDec     INTEGER,
   frequency1 FLOAT,
   frequency2 FLOAT,
   tele_array VARCHAR(4),
   baseline   VARCHAR(5),
   CONSTRAINT TelescopeConfigPK  PRIMARY KEY(configID),
   CONSTRAINT TelescopeConfigArrayCHK CHECK (tele_array IN ('1.5B','1.5C','1.5D','H168','H214','H75'))
);

CREATE TABLE IF NOT EXISTS  Epoch (
   epochID   INTEGER,
   band      INTEGER CHECK (band IN (5,8,20)),
   config    INTEGER,
   startDate DATE,
   endDate   DATE,
   CONSTRAINT Epoch_PK  PRIMARY KEY (epochID, band),
   CONSTRAINT Epoch_FK1 FOREIGN KEY (band)   REFERENCES FrequencyBand,
   CONSTRAINT Epoch_FK2 FOREIGN KEY (config) REFERENCES TelescopeConfig,
   CONSTRAINT EpochDates_CHK CHECK (startDate <= endDate)
);

CREATE TABLE IF NOT EXISTS  EpochShorthand (
   epoch INTEGER,
   band  INTEGER,
   abbrv VARCHAR(10),
   CONSTRAINT EpochShorthandPK PRIMARY KEY(epoch,band,abbrv),
   CONSTRAINT EpochShorthandFK FOREIGN KEY(epoch,band) REFERENCES Epoch
);

CREATE TABLE IF NOT EXISTS  Measurement (
   gid          VARCHAR(20),
   epoch        INTEGER,
   band         INTEGER,
   intensity    INTEGER,
   error        VARCHAR(10), -- INT does not work?
   polarisation VARCHAR(6),
   polError     VARCHAR(2),
   polFrac      VARCHAR(6),
   polAngle     VARCHAR(4),
   CONSTRAINT MeasurementPK  PRIMARY KEY (gid,epoch,band),
   CONSTRAINT MeasurementFK1 FOREIGN KEY (epoch,band) REFERENCES Epoch,
   CONSTRAINT MeasurementFK2 FOREIGN KEY (band) REFERENCES FrequencyBand,
   CONSTRAINT MeasurementFK3 FOREIGN KEY (gid) REFERENCES Galaxy
);

DROP TABLE IF EXISTS Variability;
CREATE TABLE IF NOT EXISTS Variability (
   gid       VARCHAR(20),
   epoch     INTEGER,
   band      INTEGER,
   intensity FLOAT,
   error     FLOAT,
   CONSTRAINT VariabilityPK  PRIMARY KEY (gid,epoch,band),
   CONSTRAINT VariabilityFK1 FOREIGN KEY (epoch,band) REFERENCES Epoch,
   CONSTRAINT VariabilityFK2 FOREIGN KEY (band) REFERENCES FrequencyBand,
   CONSTRAINT VariabilityFK3 FOREIGN KEY (gid) REFERENCES Galaxy
);
</pre>

In [ ]:

# 1st ensure that the schema is in place
#
table_schema = """CREATE TABLE IF NOT EXISTS  Galaxy (
   gid  VARCHAR(20),
   ra   VARCHAR(11),
   dec  VARCHAR(11),
   CONSTRAINT GalaxyPK PRIMARY KEY(gid)
)"""

try: 
    conn = psycopg2.connect(database=DATABASENAME)
    print('connected')
    pgexec (conn, table_schema, None, "Create Table Galaxy")
    conn.close();
except Exception as e:
    print(e)

## EXAMPLE SOLUTION

The following Python fragment creates the whole schema in the PostgreSQL database.
Luckily we can execute multiple SQL statements with one pgexec() call.

In [ ]:
table_schema = """CREATE TABLE IF NOT EXISTS FrequencyBand (
   band       INTEGER,
   frequency1 FLOAT,
   frequency2 FLOAT,
   CONSTRAINT FrequencyBandPK PRIMARY KEY(band),
   CONSTRAINT FrequencyBandU1 UNIQUE (frequency1),
   CONSTRAINT FrequencyBandU2 UNIQUE (frequency2)
   /* further potential integrity constraints: frequencyX >0 and < ... */
   /* alternative: FrequencyBand(band PK, frequency PK) */
);

CREATE TABLE IF NOT EXISTS  Galaxy (
   gid  VARCHAR(20),
   ra   VARCHAR(11),
   dec  VARCHAR(11),
   CONSTRAINT GalaxyPK PRIMARY KEY(gid)
);

CREATE TABLE IF NOT EXISTS  TelescopeConfig (
   configID   INTEGER,
   minDec     INTEGER,
   maxDec     INTEGER,
   frequency1 FLOAT,
   frequency2 FLOAT,
   tele_array VARCHAR(4),
   baseline   VARCHAR(5),
   CONSTRAINT TelescopeConfigPK  PRIMARY KEY(configID),
   CONSTRAINT TelescopeConfigArrayCHK CHECK (tele_array IN ('1.5B','1.5C','1.5D','H168','H214','H75'))
);

CREATE TABLE IF NOT EXISTS  Epoch (
   epochID   INTEGER,
   band      INTEGER CHECK (band IN (5,8,20)),
   config    INTEGER,
   startDate DATE,
   endDate   DATE,
   CONSTRAINT Epoch_PK  PRIMARY KEY (epochID, band),
   CONSTRAINT Epoch_FK1 FOREIGN KEY (band)   REFERENCES FrequencyBand,
   CONSTRAINT Epoch_FK2 FOREIGN KEY (config) REFERENCES TelescopeConfig,
   CONSTRAINT EpochDates_CHK CHECK (startDate <= endDate)
);

CREATE TABLE IF NOT EXISTS  EpochShorthand (
   epoch INTEGER,
   band  INTEGER,
   abbrv VARCHAR(10),
   CONSTRAINT EpochShorthandPK PRIMARY KEY(epoch,band,abbrv),
   CONSTRAINT EpochShorthandFK FOREIGN KEY(epoch,band) REFERENCES Epoch
);

CREATE TABLE IF NOT EXISTS  Measurement (
   gid          VARCHAR(20),
   epoch        INTEGER,
   band         INTEGER,
   intensity    INTEGER,
   error        VARCHAR(10), -- INT does not work?
   polarisation VARCHAR(6),
   polError     VARCHAR(2),
   polFrac      VARCHAR(6),
   polAngle     VARCHAR(4),
   CONSTRAINT MeasurementPK  PRIMARY KEY (gid,epoch,band),
   CONSTRAINT MeasurementFK1 FOREIGN KEY (epoch,band) REFERENCES Epoch,
   CONSTRAINT MeasurementFK2 FOREIGN KEY (band) REFERENCES FrequencyBand,
   CONSTRAINT MeasurementFK3 FOREIGN KEY (gid) REFERENCES Galaxy
);

DROP TABLE IF EXISTS Variability;
CREATE TABLE IF NOT EXISTS Variability (
   gid       VARCHAR(20),
   epoch     INTEGER,
   band      INTEGER,
   intensity FLOAT,
   error     FLOAT,
   CONSTRAINT VariabilityPK  PRIMARY KEY (gid,epoch,band),
   CONSTRAINT VariabilityFK1 FOREIGN KEY (epoch,band) REFERENCES Epoch,
   CONSTRAINT VariabilityFK2 FOREIGN KEY (band) REFERENCES FrequencyBand,
   CONSTRAINT VariabilityFK3 FOREIGN KEY (gid) REFERENCES Galaxy
);"""

try: 
    conn = psycopg2.connect(database=DATABASENAME)
    print('connected')
    pgexec (conn, table_schema, None, "Created New Schema")
    conn.close();
except Exception as e:
    print(e)

# STOP PLEASE. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.

## EXERCISE 5: Data Loading and Storage

Up-to this point, we have
 - analysed and modelled the given data set
 - created a corresponding relational star schema
 - read the individual CSV files into Python dictionary data structures
 - cleaned the raw data with regard to missing or inconsistent entries and data types
 
The final step is to load this cleaned data into the corresponding tables of the star schema which we defined so far.

For this to work, you probably will need to write some logic to load different parts of different data dictionaries (holding the content of CSV files) into different tables.
For example, the first few attributes of the 'main catalogue' table define the galaxy entities, hence have to go into the Galaxy table.

In [ ]:
# make sure we loaded the CSV file with the main catalogue
import csv
import pprint
data_maincatalogue = list(csv.DictReader(open('04-at20g-short-main-catalogue.csv')))

# connect to the database
import psycopg2
try: 
    conn = psycopg2.connect(database=DATABASENAME)
    print('connected')
except Exception as e:
    print("unable to connect to the database")
    print(e)

# check for any existing content of the Galaxy table
query_stmt = "SELECT COUNT(*) FROM Galaxy"
print(query_stmt)
pgquery (conn, query_stmt, None)

# Try to load data - 1001 tupels should be created
insert_stmt = """INSERT INTO Galaxy(gid, ra, dec) VALUES (%(gid)s, %(ra)s, %(dec)s)"""
galaxydata = dict()
for row in data_maincatalogue:
    galaxydata['gid'] = row['dataset']+row['objectname']
    galaxydata['ra']  = row['ra']
    galaxydata['dec'] = row['dec']
    pgexec (conn, insert_stmt, galaxydata, "galaxy inserted")

# check content of Galaxy table
query_stmt = "SELECT * FROM Galaxy"
print(query_stmt)
pgquery (conn, query_stmt, None)

conn.close();

## Your Task: Data Storage in given Database Schema

Take all the three given CSV data sets and load them into the corresponding relational tables. You will need to split some of the table data for this.

## EXAMPLE SOLUTION

For convenience, we copy here a few steps which have been done already further up in the notebook, but this way are close together to demonstrate the complete ETL process.

### Phase 1: Extract
First, we 'extract' the data from the given CSV files into Python dictionaries.

In [ ]:
import csv
data_frequencies   = list(csv.DictReader(open('04-at20g-short-frequencies.csv')))
data_observations  = list(csv.DictReader(open('04-at20g-short-observations.csv')))
data_maincatalogue = list(csv.DictReader(open('04-at20g-short-main-catalogue.csv')))
data_variabilities = list(csv.DictReader(open('04-at20g-short-variabilities.csv')))
data_epochs        = list(csv.DictReader(open('04-at20g-short-epochs.csv')))

### Phase 2: Clean
Next, we clean the data from missing data, inconsistent entries and leading/trailing spaces.
We start by introducing a utiloity function that converts numbers, but also allows 'smaller than' and 'greater than' measurement expressions - which we keep for the moment. 

In [ ]:
def int_relaxed(value):
    if value != None:
      if type(value) == str:
        value.strip();
        if value[0] == '<':
            return value;
        if value[0] == '>':
            return value;
        return int(value);
    return value;

In [ ]:
# Frequency data: convert the two frequency columns to int values - or NaN
data_frequencies = list(iter_clean(data_frequencies, 'frequency_mhz', int, DEFAULT_VALUE))
data_frequencies = list(iter_clean(data_frequencies, 'abbreviated_frequency_ghz', int, DEFAULT_VALUE))

# Clean the observation data from leading and trailing spaces around values 
# we are applying the str.strip (string strip) function to all values of 'baseline', 'array' and 'observing_dates' columns
data_observations = list(iter_clean(data_observations, 'baseline', str.strip, None))
data_observations = list(iter_clean(data_observations, 'array',    str.strip, None))
data_observations = list(iter_clean(data_observations, 'observing_dates', str.strip, None))

# Cleaning the measurement data
# we keep the '<20' notation values, but note that due to our schema choices for the Measurement table
# we need to use None (NULL) rather than NaN (not supported for integer columns)
# and we also transform all polarisation values, if not correct, to None (NULL)
data_frequencies = list(iter_clean(data_maincatalogue, 'S5',    int_relaxed, None))
data_frequencies = list(iter_clean(data_maincatalogue, 'err5',  int_relaxed, DEFAULT_VALUE))
data_frequencies = list(iter_clean(data_maincatalogue, 'P5',    int_relaxed, None))
data_frequencies = list(iter_clean(data_maincatalogue, 'P5err', int_relaxed, None))
data_frequencies = list(iter_clean(data_maincatalogue, 'M5',    int_relaxed, None))
data_frequencies = list(iter_clean(data_maincatalogue, 'M5pa',  int_relaxed, None))
data_frequencies = list(iter_clean(data_maincatalogue, 'S8',    int_relaxed, None))
data_frequencies = list(iter_clean(data_maincatalogue, 'err8',  int_relaxed, DEFAULT_VALUE))
data_frequencies = list(iter_clean(data_maincatalogue, 'P8',    int_relaxed, None))
data_frequencies = list(iter_clean(data_maincatalogue, 'P8err', int_relaxed, None))
data_frequencies = list(iter_clean(data_maincatalogue, 'M8',    int_relaxed, None))
data_frequencies = list(iter_clean(data_maincatalogue, 'M8pa',  int_relaxed, None))
data_frequencies = list(iter_clean(data_maincatalogue, 'S20',   int_relaxed, None))
data_frequencies = list(iter_clean(data_maincatalogue, 'err20', int_relaxed, DEFAULT_VALUE))
data_frequencies = list(iter_clean(data_maincatalogue, 'P20',   int_relaxed, None))
data_frequencies = list(iter_clean(data_maincatalogue, 'P20err',int_relaxed, None))
data_frequencies = list(iter_clean(data_maincatalogue, 'M20',   int_relaxed, None))
data_frequencies = list(iter_clean(data_maincatalogue, 'M20pa', int_relaxed, None))

# Cleaning the variabilities data
data_variabilities = list(iter_clean(data_variabilities, 'oct04_flux', float, DEFAULT_VALUE))
data_variabilities = list(iter_clean(data_variabilities, 'oct04_err',  float, DEFAULT_VALUE))
data_variabilities = list(iter_clean(data_variabilities, 'oct05_flux', float, DEFAULT_VALUE))
data_variabilities = list(iter_clean(data_variabilities, 'oct05_err',  float, DEFAULT_VALUE))
data_variabilities = list(iter_clean(data_variabilities, 'apr06_flux', float, DEFAULT_VALUE))
data_variabilities = list(iter_clean(data_variabilities, 'apr06_err',  float, DEFAULT_VALUE))

# Cleaning the epoch shorthand data
# we could replace all '-' with None, but we decided to do it in the load function later directly

### Phase 3: Transform and Load
For the loading phase, we first introduce another utility function which will help us retrieving individual values from postgres.

In [ ]:
def pgqueryval( conn, sqlcmd, args, silent=False ):
   """ utility function to execute some SQL query statement
       it can take optional arguments to fill in (dictionary)
       returns the first result record from the query (any more than the 1st will be ignored)
       error and transaction handling built-in """
   retval = None
   with conn:
      with conn.cursor() as cur:
         try:
            if args is None:
                cur.execute(sqlcmd)
            else:
                cur.execute(sqlcmd, args)
            retval = cur.fetchone()
         except Exception as e:
            if silent == False:
                print("db read error: ")
                print(e)
   return retval

### SubTask 1: Loading Frequencies.csv

The data of the '04-at20g-short-frequencies.csv' CSV file be loaded into the <tt>FrequencyBand</tt> table. 

The transformation step here is that dual-entries for the same frequency band will be merged into the same row, but separate columns ('frequency1' and 'frequency2').

In [ ]:
# connect to the database
import psycopg2
try: 
    conn = psycopg2.connect(database=DATABASENAME)
    print('connected')
except Exception as e:
    print("unable to connect to the database")
    print(e)

# cleanup of any potential previous left-over-rows in the table
pgexec (conn, "DELETE FROM FrequencyBand", None, "FrequencyBand cleared", True)

# load data from the frequencies dictionary
insert_stmt_FB = """INSERT INTO FrequencyBand VALUES (%(band)s, %(freq)s, NULL)"""
update_stmt_FB = """UPDATE FrequencyBand SET frequency2 = %(freq)s WHERE band=%(band)s"""

fbdata= dict()
for row in data_frequencies:
    fbdata['band'] = row['abbreviated_frequency_ghz']
    fbdata['freq'] = row['frequency_mhz']
    success = pgexec (conn, insert_stmt_FB, fbdata, "FrequencyBand loaded", True)
    if success == False:
        pgexec (conn, update_stmt_FB, fbdata, "FrequencyBand updated")

# check content of FrequencyBand table
query_stmt = "SELECT * FROM FrequencyBand"
print(query_stmt)
pgquery (conn, query_stmt, None)

conn.close();

### SubTask 2: Loading Observations.csv

The data of the '04-at20g-short-observations.csv' CSV file gets split into two separate tables: the **<tt>TelescopeConfig</tt>** and the **<tt>Epoch</tt>** tables. 

While the entries for <tt>TelescopeConfig</tt> are relative straight-forward, we need to apply more transformations to the observation data. For example, start and end dates are now separately stored and need to get extracted from the interval string of the CSV file. Also some observations are made in two epochs, in which case we also transpose the two epoch values into two separate <tt>Epoch</tt> rows.

In [ ]:
# re-connect to the database
import psycopg2
try: 
    conn = psycopg2.connect(database=DATABASENAME)
    print('connected')
except Exception as e:
    print("unable to connect to the database")
    print(e)

# cleanup of any potential previous left-overs in the two tables
delete_stmt_T = """DELETE FROM TelescopeConfig"""
delete_stmt_E = """DELETE FROM Epoch"""
pgexec (conn, delete_stmt_T, None, "TelescopeConfig cleared", True)
pgexec (conn, delete_stmt_E, None, "Epoch cleared", True)

# TRANSFORM AND LOAD data into both tables
# we first define the two insert statements for later use
insert_stmt_T = """INSERT INTO TelescopeConfig(configID, minDec, maxDec, frequency1, frequency2, tele_array, baseline) 
                   VALUES (%(id)s, %(mindec)s, %(maxdec)s, %(freq1)s, %(freq2)s, %(tarray)s, %(bline)s)"""
insert_stmt_E = """INSERT INTO Epoch(epochID, band, config, startDate, endDate) VALUES (%(id)s, %(band)s, %(confg)s, %(sdate)s, %(edate)s)"""

# next we iterate through the whole observation data and convert the raw CSV file into the correct values needed by the SQL INSERT statement
configdata= dict()
epochdata = dict()
for row in data_observations:
    # compute new config id based on extisting values + 1
    configId = pgqueryval(conn, "SELECT COALESCE(MAX(configID),0)+1 FROM TelescopeConfig", None)
    
    # transform observation data into one TelescopeConfig entry
    configdata['id']    = configId[0]
    configdata['mindec']= row['min_declination']
    configdata['maxdec']= row['max_declination']
    configdata['freq1'] = row['frequency1']
    configdata['freq2'] = row['frequency2']
    configdata['tarray']= row['array']
    configdata['bline'] = row['baseline']
    pgexec (conn, insert_stmt_T, configdata, "TelescopeConfig inserted")

    # transform observation data also into one or two Epoch entries
    startdate = row['observing_dates'][0:6]+row['observing_dates'][-5:]
    enddate   = row['observing_dates'][-11:]
    if row['frequency1'] == '4800':
        epochdata['id']    = row['epoch']
        epochdata['band']  = 5
        epochdata['confg'] = configId
        epochdata['sdate'] = startdate
        epochdata['edate'] = enddate
        pgexec (conn, insert_stmt_E, epochdata, "epoch inserted")
        epochdata['id']    = row['epoch']
        epochdata['band']  = 8
        epochdata['confg'] = configId
        epochdata['sdate'] = startdate
        epochdata['edate'] = enddate
        pgexec (conn, insert_stmt_E, epochdata, "epoch inserted")
    else:
        epochdata['id']    = row['epoch']
        epochdata['band']  = 20
        epochdata['confg'] = configId
        epochdata['sdate'] = startdate
        epochdata['edate'] = enddate
        pgexec (conn, insert_stmt_E, epochdata, "epoch inserted")

# check final content of TelescopeConfig table
query_stmt = "SELECT * FROM TelescopeConfig"
print(query_stmt)
pgquery (conn, query_stmt, None)

# check final content of Epoch table
query_stmt = "SELECT * FROM Epoch"
print(query_stmt)
pgquery (conn, query_stmt, None)

conn.close();

### SubTask 3: Load MainCatalogue.csv

Next is the task to load the '04-at20g-short-main-catalogue.csv' file into the <tt>Measurement</tt> table. 

Most of the work with the measurement data was already done before in the cleaning phase. All which is left here is to combine the first two values of the raw CSV file into the correct galaxy ID value which matches that values from the other tables.

Note that we did create already the entries in the <tt>Galaxy</tt> table in our example loading code as part of the Tutorial. Otherwise this would be part of this task here too.

The main transformation left here is a **split** and **transpose** operation:
Each entry from '04-at20g-short-main-catalogue.csv' gets split into up-to four entries:
1. An <tt>Galaxy</tt> entry
2. Between one and three entries <tt>Measurement</tt> entries, depending on which frequencies were observed

In [ ]:
# connect to the database
import psycopg2
try: 
    conn = psycopg2.connect(database=DATABASENAME)
    print('connected')
except Exception as e:
    print("unable to connect to the database")
    print(e)

# cleanup of any potential previous left-overs in the two tables
pgexec (conn, "DELETE FROM Measurement", None, "Measurement cleared", True)

# TRANSFORM AND LOAD data into the Measurement table
# we first define the insert statement for later use
insert_stmt_M = """INSERT INTO Measurement(gid, epoch, band, intensity, error, polarisation, polError, polFrac, polAngle) 
                   VALUES (%(galaxy)s, %(epoch)s, %(band)s, %(intensity)s, %(error)s, %(pol)s, %(polError)s, %(polFrac)s, %(polAngle)s)"""

# next we iterate through the measurment data and convert the raw CSV file into the correct values needed by the SQL INSERT statement
measurement = dict()
for row in data_maincatalogue:
    epoch20 = row['epoch'][0:1]
    epoch8  = row['epoch'][1:2]
    epoch5  = row['epoch'][2:3]
    measurement['galaxy']   = row['dataset']+row['objectname']
    if epoch5 != '.':
        measurement['epoch']    = epoch5
        measurement['band']     = 5
        measurement['intensity']= row['S5']
        measurement['error']    = row['err5']
        measurement['pol']      = row['P5']
        measurement['polError'] = row['P5err']
        measurement['polFrac']  = row['M5']
        measurement['polAngle'] = row['M5pa']
        pgexec (conn, insert_stmt_M, measurement, "5GHz measurement inserted")
    if epoch8 != '.':
        measurement['epoch']    = epoch8
        measurement['band']     = 8
        measurement['intensity']= row['S8']
        measurement['error']    = row['err8']
        measurement['pol']      = row['P8']
        measurement['polError'] = row['P8err']
        measurement['polFrac']  = row['M8']
        measurement['polAngle'] = row['M8pa']
        pgexec (conn, insert_stmt_M, measurement, "8GHz measurement inserted")
    if epoch20 != '.':
        measurement['epoch']    = epoch20
        measurement['band']     = 20
        measurement['intensity']= row['S20']
        measurement['error']    = row['err20']
        measurement['pol']      = row['P20']
        measurement['polError'] = row['P20err']
        measurement['polFrac']  = row['M20']
        measurement['polAngle'] = row['M20pa']
        pgexec (conn, insert_stmt_M, measurement, "8GHz measurement inserted")

# check content of Measurement table
query_stmt = "SELECT * FROM Measurement"
print(query_stmt)
pgquery (conn, query_stmt, None)

conn.close();

### SubTask 4: Load Variabilities.csv

Next is the task to load the '04-at20g-short-variabilities.csv' file into the new <tt>Variability</tt> table. 

Note that we did already clean the variabilities data before. All left here is 
1. a value transformation step that converts the intensity values to milliJanskys to match the values later from the <tt>Measurement</tt> table.
2. a **transpose** operation that stores each row of the original CSV file in up-to three rows of the <tt>Variability</tt>? table, one for each frequency band.
3. if there is no variablity value for one of those bands - after our initial cleaning step represented by NaN - we simply ignore those entries and do not store them at all in the database

In [ ]:
import psycopg2
import numpy as np

# connect to the database
try: 
    conn = psycopg2.connect(database=DATABASENAME)
    print('connected')
except Exception as e:
    print("unable to connect to the database")
    print(e)

# cleanup of any potential previous left-overs in the Variability table
pgexec (conn, "DELETE FROM Variability", None, "Variability cleared", True)

# TRANSFORM AND LOAD data into the Variability table
# we first define the insert statement for later use
insert_stmt_V = """INSERT INTO Variability(gid, epoch, band, intensity, error) 
                   VALUES (%(galaxy)s, %(epoch)s, %(band)s, %(intensity)s, %(error)s)"""

# next we iterate through the variability data and convert the CSV file into the correct values needed by the SQL INSERT statement
variability = dict()
for row in data_variabilities:
    variability['galaxy']   = row['source']
    if not( np.isnan(row['oct04_flux']) ):
        variability['epoch']    = 1
        variability['band']     = 20
        variability['intensity']= row['oct04_flux'] * 1000; # convert Jansky value to milliJansky
        variability['error']    = row['oct04_err']
        pgexec (conn, insert_stmt_V, variability, "20GHz variability inserted")
    if not( np.isnan(row['oct05_flux']) ):
        variability['epoch']    = 2
        variability['band']     = 20
        variability['intensity']= row['oct05_flux'] * 1000; # convert Jansky value to milliJansky
        variability['error']    = row['oct05_err']
        pgexec (conn, insert_stmt_V, variability, "20GHz variability inserted")
    if not( np.isnan(row['apr06_flux']) ):
        variability['epoch']    = 3
        variability['band']     = 20
        variability['intensity']= row['apr06_flux'] * 1000; # convert Jansky value to milliJansky
        variability['error']    = row['apr06_err']
        pgexec (conn, insert_stmt_V, variability, "20GHz variability inserted")

# check content of Variability table
query_stmt = "SELECT * FROM Variability"
print(query_stmt)
pgquery (conn, query_stmt, None)

conn.close();

### SubTask 5: Load Epochs.csv

The final task is to load the '04-at20g-short-main-epochs.csv' file into the <tt>EpochsShorthand</tt> table. 

This is again an example of an **transpose** transformation, in that each row of the original CSV file becomes up-to three rows in the database table. For example, the CSV entry (1, oct04, nov04) will be loaded into three rows that specify the three different date-names used for the three measurement bands 20Ghz, 8Ghz and 5Ghz: 

    (1, 20, oct04)
    (1,  5, oct04)
    (1,  8, oct04) 

Additionally, we also ignore all '-' entries as those represent non-existing data, and hence we simply do no create any entry in our database for it.

In [ ]:
# connect to the database
import psycopg2
try: 
    conn = psycopg2.connect(database=DATABASENAME)
    print('connected')
except Exception as e:
    print("unable to connect to the database")
    print(e)

# cleanup of any potential previous left-overs in the EpochShorthand table
pgexec (conn, "DELETE FROM EpochShorthand", None, "EpochShorthand cleared", True)

# TRANSFORM AND LOAD data into the EpochShorthand table
# we first define the insert statement for later use
insert_stmt_ES = """INSERT INTO EpochShorthand(epoch, band, abbrv) 
                         VALUES (%(epoch)s, %(band)s, %(abbrv)s)"""

# we convert the raw CSV file into the correct values needed by the SQL INSERT statement
epochdata = dict()
for row in data_epochs:
    epochdata['epoch'] = row['epoch']
    if row['date_20ghz'] != '-':
        epochdata['band']  = 20
        epochdata['abbrv'] = row['date_20ghz']
        pgexec (conn, insert_stmt_ES, epochdata, "20GHz epoch shorthand inserted")
    if row['date_5or8gHz'] != '-':
        epochdata['band']  = 8
        epochdata['abbrv'] = row['date_5or8gHz']
        pgexec (conn, insert_stmt_ES, epochdata, "8GHz epoch shorthand inserted")
        epochdata['band']  = 5
        epochdata['abbrv'] = row['date_5or8gHz']
        pgexec (conn, insert_stmt_ES, epochdata, "5GHz epoch shorthand inserted")

# check content of EpochShorthand table
query_stmt = "SELECT * FROM EpochShorthand"
print(query_stmt)
pgquery (conn, query_stmt, None)

conn.close();

# End of Tutorial. Many Thanks.